<a href="https://colab.research.google.com/github/MuhammadHelmyOmar/Authorship-Identification/blob/main/topic_anlalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Loading Data

In [5]:
from google.colab import drive
import pandas as pd
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
file_path = '/content/drive/MyDrive/Colab Notebooks/AraGenEval/train_data_splitted.csv'

train_data = pd.read_csv(file_path)
display(train_data.head())

,original_id,text_in_author_style,author,text_size,chunk_index,is_split,cleaned_text
0,5843,"هذه الكتب التي أصدرتُها منذ بدأت كتابة باب ""من...",يوسف إدريس,1921,0,False,"هذه الكتب التي أصدرتها منذ بدأت كتابة باب ""من ..."
1,5844,صعب جِدًّا في ظل هذا التقسيم الإرهابي أن أقول ...,يوسف إدريس,1616,0,False,صعب جدا في ظل هذا التقسيم الإرهابي أن أقول إن ...
2,5845,كان ممكنًا أن أتفادى هذا\n\nالموضوع الشائك، وأ...,يوسف إدريس,1913,0,False,كان ممكنا أن أتفادى هذا\n\nالموضوع الشائك، وأذ...
3,5846,ومن المُمكن للعراق وحده أن يَجتثَّه ويقضي على ...,يوسف إدريس,2047,0,False,ومن الممكن للعراق وحده أن يجتثه ويقضي على الهج...
4,5847,كنتُ أريد أن أتناول موضوعًا من هذه المواضيع.\n...,يوسف إدريس,2022,0,False,كنت أريد أن أتناول موضوعا من هذه المواضيع.\n\n...


# Topic Modeling

In [1]:
!pip install -qU bertopic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.6/150.6 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 115.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 67.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 101.7 MB/s eta 0:00:00


## Embed Articles

In [2]:
from sentence_transformers import SentenceTransformer
import pickle

In [3]:
model_id = "Omartificial-Intelligence-Space/GATE-AraBert-v1"
embedding_model = SentenceTransformer(model_id, device="cuda:0")

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/666 [00:00<?, ?B/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [8]:
articles_embeddings = embedding_model.encode(train_data['text_in_author_style'].values, show_progress_bar=True, batch_size=64)

print(articles_embeddings.shape)

Batches:   0%|          | 0/515 [00:00<?, ?it/s]

(32941, 768)


In [10]:
# Store data and embeddings
# with open("/content/drive/MyDrive/Colab Notebooks/AraGenEval/authorship_embeddings_GATE-AraBert-v1.pkl", "wb") as fOut:
#   pickle.dump({'embeddings': articles_embeddings},
#               fOut,
#               protocol = pickle.HIGHEST_PROTOCOL)

In [11]:
# Load data and embeddings
with open("/content/drive/MyDrive/Colab Notebooks/AraGenEval/authorship_embeddings_GATE-AraBert-v1.pkl", 'rb') as fIn:
  articles_embeddings = pickle.load(fIn)['embeddings']

# Gemi

Replace the placeholder path with the actual path to your data file in Google Drive.